<a href="https://colab.research.google.com/github/yc386/orthrus_metaproteomics/blob/main/Orthrus_stable_v100/orthrus_stable_v100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?export=view&id=19rmmQI1H2nIqgU598WROTcUNhOUoXcBP' width='400px' align='right'>

# **Readme**

---
[Orthrus](https://www.biorxiv.org/content/10.1101/2024.11.15.623814v1) 🐾 is a hybrid, two-software pipeline that integrates [Casanovo](https://github.com/Noble-Lab/casanovo) (an AI transformer) with [Sage](https://github.com/lazear/sage) (a fast database search engine with advanced features like retention time alignment and machine learning-based rescoring).

Designed to handle large search spaces in metaproteomics and palaeoproteomics, Orthrus leverages *de novo* sequencing to define sample-specific databases, and uses probability ranking and conventional database searching to control FDRs (false discovery rates).

Orthrus is optimised for online use on Google Colab 🥳

# **Quick start**❗️
1. Before walking the dog, please change the runtime type to GPU (A100, L4, or T4. A100 most efficient but T4 is free)
2. Click the folder image 🗂️ on the left and mount your Google drive (permission pending)
3. Click `File` (top left) to save a local copy
4. **Run the `Install everything, will automatically restart` cell first** and wait for restarting, to resolve the numpy+pandas version conflicts. Casanovo 4x and Mokapot require numpy 1x
5. Choose, Casanovo, Sage, and Mokapot configurations. Then **from the `Configure Casanovo` cell, cick `Runtime` -> `Run cell and below`**

In [1]:
#@title Install everything, will **automatically restart** to resolve version conflicts

import os, sys, subprocess, time, IPython
from pathlib import Path
if not Path("Orthrus_READY").exists():
    print("installing conda and packages📦 ⬇️")
    #Sage version 0.14.7
    !wget -q https://github.com/lazear/sage/releases/download/v0.14.7/sage-v0.14.7-x86_64-unknown-linux-gnu.tar.gz
    !tar -xzf sage-v0.14.7-x86_64-unknown-linux-gnu.tar.gz && rm sage-v0.14.7-x86_64-unknown-linux-gnu.tar.gz
    pip_packages = ["casanovo==4.3.0", "biopython==1.85", "pyteomics==4.7.5", "mokapot==0.10.0", "numpy==1.26.4", "pandas==2.1.4", "xgboost==3.0.4"]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade",
                           *pip_packages])
    Path("Orthrus_READY").touch()
else:
    print("Environment already prepared.")

msg = "Restarting 🫨 ➡️ Don't click any Google prompts"
print(msg, flush=True)
time.sleep(0.5)
IPython.Application.instance().kernel.do_shutdown(restart=True)

installing conda and packages📦 ⬇️
Restarting 🫨 ➡️ Don't click any Google prompts


{'status': 'ok', 'restart': True}

In [ ]:
#@title Configure `Casanovo`
#@markdown **`Casanovo` inputs**
folder_path=""#@param {type:"string"}
#@markdown - a folder contains single or multiple `.mzML` or `.mgf` files for `Casanovo`. Please check only _ (underscore) and no other special characters or space in a file name. Ensure all instrument files in a single folder
file_type="mzML" #@param ["mzML", "mgf"]
#@markdown - use the drop-down menu to choose the instrument file type

use_default = False #@param {type:"boolean"}
#@markdown - use the default model + configuration yaml from `Casanovo` github repo, may take time to download

#@markdown **Advanced Options (user provided model + configuration yaml)**

model = "/content/drive/MyDrive/casanovo/models/casanovo4/ckpt/casanovo_massivekb_worker1.ckpt" #@param {type:"string"}
#@markdown - a `.ckpt` trained model (check point)
config = "/content/drive/MyDrive/casanovo/models/casanovo4/yaml/config_420.yaml" #@param {type:"string"}
#@markdown - a `.yaml` configuration file (see config_420_precursor_7_ppm.yaml)

#@markdown **Inputs for converting Casanovo results to a `.fasta`**
use_SwissProt = True #@param {type:"boolean"}
#@markdown - use the latest, reviewed SwissProt form the UniProt FTP
database_path=""#@param {type:"string"}
#@markdown - path to a user-defined database (`.fasta`)

In [ ]:
#@title Configure `SAGE`
json_file_path = '/content/drive/MyDrive/casanovo/sage/config_general_MQ_fixed_CAM_v1.json' #@param {type:"string"}
#@markdown - a configuration `.json` file (see config_general_MQ_fixed_CAM_v1.json)
enzyme = "KR" #@param {type:"string"}
#@markdown **`SAGE` PTM plus**
#@markdown - Default `Sage` contains CAM (fixed) (+57.021464) + variable mods: Oxidation(M) (+15.994915), Deamidation(NQ) (+0.984016)
#@markdown - PTM plus up to 5 variable mods and CAM (cysteine carbamidomethylation) can be turned off
#@markdown - PTM mass can be any decimals
use_PTM_plus = True #@param {type:"boolean"}
static_CAM = True #@param {type:"boolean"}
max_variable_mods= 3 #@param {type:"number"}
#@markdown - please note `SAGE` only allow max 3 variable mods per PSM
missed_cleavages= 2 #@param {type:"number"}
AA_1 = "M" #@param ["None", "[","]","A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
AA_1_mod = 15.9949 #@param {type:"number"}
AA_2 = "P" #@param ["None", "[","]","A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
AA_2_mod = 15.9949 #@param {type:"number"}
AA_3 = "N" #@param ["None", "[","]","A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
AA_3_mod = 0.984016 #@param {type:"number"}
AA_4 = "Q" #@param ["None", "[","]","A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
AA_4_mod = 0.984016 #@param {type:"number"}
AA_5 = "None" #@param ["None", "[","]","A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
#@markdown - [ = n-terminal
AA_5_mod = 	42.010565 #@param {type:"number"}


In [ ]:
#@title Configure Mokapot

joint_modelling= True #@param {type:"boolean"}
#@markdown - a joint model for low abundance samples, unclick for a separate model per experiment
default_Percolator=True #@param {type:"boolean"}
#@markdown - Python implementation of the Percolator SVM model
#@markdown - the other option is the [XGBoost model](https://pubs.acs.org/doi/10.1021/acs.jproteome.0c01010)

In [1]:
#@title import functions

import re, glob, json, requests, gzip, shutil, os, subprocess, sys
import pandas as pd
import numpy as np
from pyteomics import mztab
from numpy import string_
from joblib import Parallel, delayed
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

'''
parse a .mztab file using pyteomics
add naked sequences (without PTMs) & the sequence length
input=path to .mztab file
output=pandas dataframe

'''
pattern = re.compile(r'(.\d*\.?\d+)')

def prep_mztab(mztab_path):
  m = mztab.MzTab(mztab_path)
  df = m.spectrum_match_table
  if df is None or df.empty:
    raise ValueError(f"{mztab_path} is empty")
  if 'sequence' not in df.columns:
    raise KeyError(f"'sequence' column is missing in the file: {mztab_path}")
  df.reset_index(drop=True)
  df1 = df.assign(sequence_naked=df['sequence'].str.replace(pattern, '', regex=True))
  df2= df1.assign(nAA=df1['sequence_naked'].str.len())
  df3=df2.sort_values(by='sequence_naked').drop_duplicates(subset='sequence_naked', keep="first").reset_index(drop=True)
  return df3

'''
parse a .fasta file using biopython
add UniProt ID e.g. P02754
input=path to .fasta file
output=pandas dataframe

'''

def fasta_to_df(fasta_file):

  data = []

  for record in SeqIO.parse(fasta_file, "fasta"):

    protein_id = record.id
    description = record.description
    sequence = str(record.seq)
    if not sequence:
      raise ValueError(f"Record with ID '{protein_id}' has no sequence in the fasta file.")

    data.append((protein_id, description, sequence))

  df = pd.DataFrame(data, columns=["Protein_ID", "Description", "Sequence"])
  df1=df.assign(UniProt_ID=df['Protein_ID'].str.split('|').str[1])

  return df1


'''
filter a Casanovo output file based on the maximum value below 0
search_engine_score[1] is a score assigned to each prediction by Casanovo, max=1,
if negative then outside the mass tolerance
input=pandas dataframe
output=pandas dataframe

'''

def casa_filter (df):
  np_array = df['search_engine_score[1]'].to_numpy()
  max_below_zero = np_array[np_array < 0].max()
  df1=df[df['search_engine_score[1]']>=max_below_zero]
  return df1


#prepare overlapping sequence tags for string matching
def get_seq_tags (sequence, k):
  return set(sequence[i:i+k] for i in range(len(sequence) - k + 1))


'''
match de novo-based tags with database tags
I=L in a reference database
inputs=path to fasta, filtered casanovo output dataframe, tag size=k, chunk size=10000 for processing
output=pandas dataframe

'''

def matching_count_v5 (df, df1, k, chunk_size=10000):

  sequence_set = get_seq_tags(''.join(chain.from_iterable(df1['sequence_naked'].astype(str))), k)
  print(f"📝 {len(sequence_set)} tags regenerated. Starting matching...")
  result_df = pd.DataFrame()
  for start in range(0, len(df), chunk_size):
    chunk = df.iloc[start:start+chunk_size].copy()
    chunk['seq_tags'] = chunk['Sequence'].astype(str).str.replace('I', 'L').apply(lambda x: get_seq_tags(x, k))
    chunk['matched_count'] = chunk['seq_tags'].apply(lambda seq_tags: len(seq_tags & sequence_set))
    chunk = chunk.assign(matched=chunk['matched_count'].apply(lambda x: 1 if x >= 2 else 0))
    result_df = pd.concat([result_df, chunk], ignore_index=True)
  total_matches=result_df['matched_count'].sum()
  print(f"Completed! {total_matches} matched 👍 ")
  return result_df

#get tryptic peptides per database entry
def count_tryptic_peptides(sequence):
  pattern=r'(?<=[KR])'

  peptides = re.split(pattern, sequence)

  filtered_peptides = [peptide for peptide in peptides if len(peptide) >= 6]

  return len(filtered_peptides)

#prepare a dataframe for NB classification
def prep_Bayes (df):
  print('🧑‍💻 Start Bayes probabilistic ranking...')
  df1=df.assign(length=df['Sequence'].astype(str).str.len(),
                 tryptic_count=df['Sequence'].apply(count_tryptic_peptides),
                 tag_count=df['seq_tags'].apply(len))
  df2=df1.assign(SAF=df1['matched_count']/df1['length'],
                 try_ratio=df1['tryptic_count']/df1['tag_count']
                 )
  return df2


'''
ranking matched proteins based on SAF and try_ratio
values normalised before NB classification
most probable matches (>= 95%) are shortlised

'''
def get_bayes_ranking_test (df, threshold=0.95):
  m=prep_Bayes(df)
  required_columns = {'SAF', 'try_ratio', 'matched'}
  if not required_columns.issubset(m.columns):
    missing_cols = required_columns - set(m.columns)
    raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
  m1=m[m['tag_count']>0]
  X = m1[['SAF', 'try_ratio']].to_numpy()
  y = m1['matched'].to_numpy()
  scaler = MinMaxScaler()
  X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(*X.shape)
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=7)
  gnb = GaussianNB()
  gnb.fit(X_train, y_train)
  y_pred = gnb.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  f1=f1_score(y_test, y_pred)
  print(f"✅ Gaussian Naive Bayes model ▶️ accuracy:{accuracy:.4f}, precision:{precision:.4f}, f1:{f1:.4f}")
  whole_pred=gnb.predict(X_scaled)
  class_probabilities = gnb.predict_proba(X_scaled)
  m2=m1.assign(pred=class_probabilities[:, 1])
  m3=m2[m2['pred']>=threshold]
  return m3


#combine previous functions together to output a shortlisted .fasta

def matching_ranking_to_fasta_v5 (mztab_path, fasta_df):
  p=prep_mztab(mztab_path)
  p1=casa_filter(p)
  k=int(p1['nAA'].median())
  m=matching_count_v5 (fasta_df, p1, k, chunk_size=10000)
  m1=get_bayes_ranking_test (m)
  seq_records = []
  for index, row in m1.iterrows():
    header_id = f"{row['Description']}"
    sequence = Seq(row['Sequence'])
    description = ""
    seq_record = SeqRecord(sequence, id=header_id, description=description)
    seq_records.append(seq_record)

  output_fasta_filepath = mztab_path.replace('.mztab', '_matched.fasta')

  with open(output_fasta_filepath, 'w') as output_file:
    SeqIO.write(seq_records, output_file, 'fasta')
  print(f"🎊 Number of protein entries in the output fasta: {m1.shape[0]}")



#generate a de novo-first, experiment-specific .fasta for each input
def process_all_mztab_files_v2 (folder_path, database_path):
    mztab_filepaths = glob.glob(f"{folder_path}/*.mztab")
    print(f"🗂️ {len(mztab_filepaths)} file(s) collecting from {folder_path}...")
    fas=fasta_to_df(database_path)
    fasta_df=pd.DataFrame.from_dict(fas)
    print(f"⬆️ {database_path} loaded")
    print(f"📤 No. of proteins in the reference fasta: {fasta_df.shape[0]}")

    for mztab in mztab_filepaths:
      print(f"🚀 Processing file: {mztab}")
      matching_ranking_to_fasta_v5 (mztab, fasta_df)

'''
Organise mztabs and instrument files in the same folder

'''

def organise_files (directory):
    print("Organising files...")

    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return



    MS2_files = glob.glob(os.path.join(directory, f'*.{file_type}'))

    for MS2 in MS2_files:

        base_name = os.path.splitext(os.path.basename(MS2))[0]

        new_folder_path = os.path.join(directory, base_name)

        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
            print(f"Created folder: {new_folder_path}")
        else:
            print(f"Folder already exists: {new_folder_path}")


        MS2_path = os.path.join(new_folder_path, os.path.basename(MS2))
        if not os.path.exists(MS2_path):
            shutil.move(MS2, new_folder_path)
            print(f"Moved {MS2} to {new_folder_path}")
        else:
            print(f"MS2 file already exists in the destination: {MS2_path}")


        fasta_filename = f"{base_name}_matched.fasta"
        fasta_file = os.path.join(directory, fasta_filename)


        if os.path.exists(fasta_file):
            new_fasta_path = os.path.join(new_folder_path, fasta_filename)
            if not os.path.exists(new_fasta_path):
                shutil.move(fasta_file, new_folder_path)
                print(f"Moved {fasta_file} to {new_folder_path}")
            else:
                print(f".fasta file already exists in the destination: {new_fasta_path}")
        else:
            print(f"No matching .fasta file found for {base_name}")


"""Generate and save a Sage configuration .json file."""

def get_sage_config(json_file_path, peak_folder, static_mods, new_mods,
                    missed_cleavages, enzyme,
                    min_len, max_len, max_variable_mods,
                    output_config_path):

    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

        peak_files = glob.glob(peak_folder)
        print(f"🗂️ {len(peak_files)} file(s) collected from {peak_folder}")

        json_data['mzml_paths'] = peak_files
        json_data['database']['static_mods'] = static_mods
        json_data['database']['variable_mods'] = new_mods
        json_data['database']['enzyme']['missed_cleavages'] = missed_cleavages
        json_data['database']['enzyme']['cleave_at']= enzyme
        json_data['database']['enzyme']['min_len'] = min_len
        json_data['database']['enzyme']['max_len'] = max_len
        json_data['database']['max_variable_mods'] = max_variable_mods
        json_data['database']['decoy_tag'] = "rev_"
        json_data['database']['generate_decoys'] = True

    with open(output_config_path, 'w') as f:
        json.dump(json_data, f, indent=4)

In [ ]:
#@title Run Casanovo

if use_default:
  folder = glob.glob(f"{folder_path}/*.{file_type}")
  for instrument_file in folder:
    output_path=instrument_file.replace(f".{file_type}", ".mztab")
    ! casanovo sequence {instrument_file} -v info -o {output_path}

else:
  folder = glob.glob(f"{folder_path}/*.{file_type}")
  for instrument_file in folder:
    output_path=instrument_file.replace(f".{file_type}", ".mztab")
    ! casanovo sequence {instrument_file} -m {model} -c {config} -v info -o {output_path}

In [ ]:
#@title Convert `Casanovo` results to .fasta per experiment

if use_SwissProt:
  url = "https://ftp.uniprot.org/pub/databases/uniprot/knowledgebase/complete/uniprot_sprot.fasta.gz"
  output_file = "uniprot_sprot.fasta.gz"
  decompressed_file = "uniprot_sprot.fasta"
  response = requests.get(url, stream=True)
  if response.status_code == 200:
    with open(output_file, 'wb') as f:
      shutil.copyfileobj(response.raw, f)
    print(f"{output_file} downloaded successfully.")
  else:
    print(f"Failed to download {output_file}, status code: {response.status_code}")
  with gzip.open(output_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
      shutil.copyfileobj(f_in, f_out)
  sprot_path="uniprot_sprot.fasta"
  process_all_mztab_files_v2(folder_path, sprot_path)

else:
  process_all_mztab_files_v2(folder_path, database_path)

In [ ]:
#@title Run Sage

organise_files(folder_path)
folder_path = folder_path
#colab version
sage_path="/content/sage-v0.14.7-x86_64-unknown-linux-gnu/sage"

if use_PTM_plus:
    AAs = [AA_1, AA_2, AA_3, AA_4, AA_5]
    mods = [AA_1_mod, AA_2_mod, AA_3_mod, AA_4_mod, AA_5_mod]
    PTMs = {}

    for AA, mod in zip(AAs, mods):
        if AA != "None":
            PTMs[AA] = [mod]

    big_folder = glob.glob(f"{folder_path}/*")
    for folder in big_folder:
        if not os.path.isdir(folder):
            continue

        mzml_files = glob.glob(f"{folder}/*.{file_type}")
        if not mzml_files:
            continue

        peak_path = mzml_files[0]
        output_json = peak_path.replace(f".{file_type}", '.json')

        json_file_path = json_file_path
        missed_cleavages = missed_cleavages
        enzyme = enzyme
        min_len = 6
        max_len = 30
        max_variable_mods = max_variable_mods
        static_mods = {"C": 57.021464} if static_CAM else {}

        get_sage_config(
            json_file_path, peak_path, static_mods, PTMs,
            missed_cleavages, enzyme,
            min_len, max_len,
            max_variable_mods, output_json
        )

        fasta_files = glob.glob(f"{folder}/*.fasta")
        if not fasta_files:
            continue

        fasta_path = fasta_files[0]
        # path to Sage binary
        ! {sage_path} {output_json} --fasta {fasta_path} \
            --write-pin --output_directory {folder}

else:
    big_folder = glob.glob(f"{folder_path}/*")
    for folder in big_folder:
        if not os.path.isdir(folder):
            continue

        mzml_files = glob.glob(f"{folder}/*.{file_type}")
        if not mzml_files:
            continue

        peak_path = mzml_files[0]
        output_json = peak_path.replace(f".{file_type}", '.json')
        json_file_path = json_file_path
        missed_cleavages = 2
        enzyme = enzyme
        min_len = 6
        max_len = 30
        max_variable_mods = 5
        static_mods = {"C": 57.021464}
        new_mods = {"M": [15.994915], "N": [0.984016], "Q": [0.984016]}

        get_sage_config(
            json_file_path, peak_path, static_mods, new_mods,
            missed_cleavages, enzyme,
            min_len, max_len,
            max_variable_mods, output_json
        )

        fasta_files = glob.glob(f"{folder}/*.fasta")
        if not fasta_files:
            continue

        fasta_path = fasta_files[0]
        # path to Sage binary
        ! {sage_path} {output_json} --fasta {fasta_path} \
            --write-pin --output_directory {folder}


In [ ]:
#@title Brew Mokapot

import mokapot
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import glob

"""
XGBoost schema from Fondrie & Noble (2021).
A non-linear XGBoost seems to be better for rescoring open search results.
"""

#from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import os


grid = {
    "scale_pos_weight": np.logspace(0, 2, 3),
    "max_depth": [1, 3, 6],
    "min_child_weight": [1, 10, 100],
    "gamma": [0, 1, 10],
}


xgb_mod = GridSearchCV(
    XGBClassifier(),
    param_grid=grid,
    n_jobs=1,
    cv=3,
    scoring="roc_auc",
)

"""Recursively find all .pin files in the given folder."""
def get_all_pin_files(folder_path):
    psm_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pin'):
                full_path = os.path.join(root, file)
                psm_files.append(full_path)
    return psm_files


if joint_modelling:
     psm_files = get_all_pin_files(folder_path)
     if default_Percolator:
        svm = mokapot.PercolatorModel()
        psm_list = mokapot.read_pin(psm_files)
        results, models = mokapot.brew(psm_list, svm)
        result_files = results.to_txt(folder_path)
     else:
        mod = mokapot.Model(xgb_mod)
        psm_list = mokapot.read_pin(psm_files)
        results, models = mokapot.brew(psm_list, mod)
        result_files = results.to_txt(folder_path)

else:
    big_folder = sorted(glob.glob(f"{folder_path}/*"))

    for folder in big_folder:
        if not os.path.isdir(folder):
            continue

        print(f"Processing folder: {folder}")
        pin_files = glob.glob(f"{folder}/*.pin")

        if not pin_files:
            print(f"No .pin files found in {folder}. Skipping...")
            continue

        pin = pin_files[0]

        if default_Percolator:
            svm = mokapot.PercolatorModel()
            psm_list = mokapot.read_pin(pin)
            results, models = mokapot.brew(psm_list, svm)
            result_files = results.to_txt(folder)
        else:
            mod = mokapot.Model(xgb_mod)
            psm_list = mokapot.read_pin(pin)
            results, models = mokapot.brew(psm_list, mod)
            result_files = results.to_txt(folder)